<a href="https://colab.research.google.com/github/mojassameh/Exercises/blob/master/categorical_embeding_test_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
# ! pip install shap
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import shap
from sklearn.preprocessing import StandardScaler

data,labels = shap.datasets.adult(display=True)
data.select_dtypes('category').columns

Index(['Workclass', 'Marital Status', 'Occupation', 'Relationship', 'Race',
       'Sex', 'Country'],
      dtype='object')

In [0]:
models = []
inputs = []

In [0]:
cat_features = data.select_dtypes('category').columns

In [0]:
num_features = data.select_dtypes('number').columns

In [0]:
for col in cat_features:
    encoder = LabelEncoder()
    data[col] = encoder.fit_transform(data[col])

In [0]:
for cat in cat_features:
  vocab_size = data[cat].nunique()
  input = tf.keras.Input(shape=(1,),name='input_'+'_'.join(cat.split(' ')))
  inputs.append(input)
  embed = tf.keras.layers.Embedding(vocab_size,200,\
                                          trainable=True,embeddings_initializer=tf.initializers.RandomNormal)(input)
  embed_rehsaped =tf.keras.layers.Reshape(target_shape=(200,))(embed)
  models.append(embed_rehsaped)

In [0]:
num_input = tf.keras.layers.Input(shape=(len(num_features)),\
                                      name='input_number_features')
inputs.append(num_input)
models.append(num_input)

In [0]:
merge_models= tf.keras.layers.concatenate(models)
pre_preds = tf.keras.layers.Dense(1000)(merge_models)
pre_preds = tf.keras.layers.BatchNormalization()(pre_preds)
pre_preds = tf.keras.layers.Dense(1000)(pre_preds)
pre_preds = tf.keras.layers.BatchNormalization()(pre_preds)

In [0]:
pred = tf.keras.layers.Dense(1,activation='sigmoid')(pre_preds)
model_full = tf.keras.models.Model(inputs= inputs,\
                                    outputs =pred)
model_full.compile(loss=tf.keras.losses.binary_crossentropy,\
                    metrics=['accuracy'],
                    optimizer='adam')

In [0]:
train = data.copy()
scaler = StandardScaler()
train[num_features] = scaler.fit_transform(train[num_features])
input_dict= {
    'input_Workclass':train[cat_features[0]],
    "input_Marital_Status":train[cat_features[1]],
    "input_Occupation":train[cat_features[2]],
    "input_Relationship":train[cat_features[3]],
    "input_Race":train[cat_features[4]],
    "input_Sex":train[cat_features[5]],
    "input_Country":train[cat_features[6]],
    "input_number_features": train[num_features]
}

In [0]:
model_full.fit(input_dict,labels*1,epochs=50,batch_size=64,class_weight={0:0.5,1:0.5})

Epoch 1/50
509/509 [==============================] - 20s 38ms/step - loss: 0.2034 - accuracy: 0.8269
Epoch 2/50
509/509 [==============================] - 20s 39ms/step - loss: 0.1725 - accuracy: 0.8369
Epoch 3/50
509/509 [==============================] - 20s 39ms/step - loss: 0.1714 - accuracy: 0.8404
Epoch 4/50
509/509 [==============================] - 19s 38ms/step - loss: 0.1702 - accuracy: 0.8392
Epoch 5/50
509/509 [==============================] - 19s 37ms/step - loss: 0.1673 - accuracy: 0.8442
Epoch 6/50
509/509 [==============================] - 19s 38ms/step - loss: 0.1672 - accuracy: 0.8444
Epoch 7/50
509/509 [==============================] - 19s 38ms/step - loss: 0.1670 - accuracy: 0.8432
Epoch 8/50
509/509 [==============================] - 19s 38ms/step - loss: 0.1660 - accuracy: 0.8451
Epoch 9/50
509/509 [==============================] - 19s 38ms/step - loss: 0.1658 - accuracy: 0.8462
Epoch 10/50
509/509 [==============================] - 19s 38ms/step - loss: 0.166